In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce
import os
import random
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import pickle

In [6]:
def RandomForest(data):
    data_folder = 'Data/'
    year = data["Yapım Yılı"]
    file_name = f"{year}.csv"
    file_path = os.path.join(data_folder, file_name)
    
    
    new_df = pd.DataFrame([data])
    encoder2 = ce.OrdinalEncoder(cols=['Dizi İsimleri','Platform','Yapım Şirketi'])
    new_df_encoded = encoder2.fit_transform(new_df)
   
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        
        df.drop('Yönetmen', axis=1, inplace=True)
        df.drop('Senarist', axis=1, inplace=True)
        df.drop('Başroller', axis=1, inplace=True)
        df.drop('Besteci', axis=1, inplace=True)
        df.drop('Türler', axis=1, inplace=True)
        df.drop('Unnamed: 0', axis=1, inplace=True)
        df.drop('Unnamed: 0.1', axis=1, inplace=True)
        df = df.reset_index(drop=True)
        
        X = df.drop(['Sonuç'], axis=1)
        y = df['Sonuç']
        
        encoder = ce.OrdinalEncoder(cols=['Dizi İsimleri','Platform','Yapım Şirketi'])
        X_encoded = encoder.fit_transform(X)
        
        class_counts = y.value_counts()
        min_class_samples = class_counts.min()
        
        test_indices = []
        for c in y.unique():
            class_indices = df[df['Sonuç'] == c].index.tolist()
            if len(class_indices) >= min_class_samples:
                random_indices = random.sample(class_indices, min_class_samples)
            else:
                random_indices = random.choices(class_indices, k=min_class_samples)
            test_indices.extend(random_indices)
        
        X_test = X_encoded.loc[test_indices]
        y_test = y.loc[test_indices]
        X_train, _, y_train, _ = train_test_split(X_encoded.drop(test_indices), y.drop(test_indices), test_size=0.2, random_state=42)
        
        model = RandomForestClassifier()
        
        model.fit(X_train, y_train)
        
        # Test edilen verileri göster
        test_data = df.loc[test_indices]
        #print(f"Test data for year {year}:\n{test_data}")
        
        accuracy = model.score(X_test, y_test)
        print(f"Accuracy for year {year}: {accuracy}")
        prediction = model.predict(new_df_encoded)
        print(f"Random Forest Prediction for given data: {prediction}")
    
    
def AdaBoost(data):
    new_df = pd.DataFrame([data])
    encoder2 = ce.OrdinalEncoder(cols=['Dizi İsimleri','Platform','Yapım Şirketi'])
    new_df_encoded = encoder2.fit_transform(new_df)
    year = data["Yapım Yılı"]
    with open('AdaBoost_2020.pkl', 'rb') as file:
        model = pickle.load(file)
    prediction = model.predict(new_df_encoded)
    print(f"Ada Boost Prediction for given data: {prediction}")
    
    
def DecisionTree(data):
    year = data["Yapım Yılı"]

In [10]:
def predict(data):
    RandomForest(data)
    AdaBoost(data)
    DecisionTree(data)
    
data = {'Dizi İsimleri': 'Yeni Dizi','Yapım Yılı':2018,
            'Platform': 'Kanal D','Aile':0,'Dram':1,'Komedi':1,'Aksiyon':0,'Polisiye':0,'Tarihi':0,'Romantik':0,
            'Yapım Şirketi': 'Ay Yapım','Gösterim Süresi':90,
            'Başrollerin Sezon Ortalaması':1.87,
            'Yönetmen Sezon Ortalaması':2.1,'Senarist Sezon Ortalaması':2,'Ödüllü Yönetmen':1,'Ödüllü Aktör':0,'İhraç':0,
            'Ödüllü Dizi':0
            }
predict(data)

Accuracy for year 2018: 0.6
Random Forest Prediction for given data: ['Kısmen Başarılı']
Ada Boost Prediction for given data: ['Kısmen Başarılı']
